Usando a base de dados da [Pesquisa Nacional de Saúde do Escolar - PeNSE](https://www.ibge.gov.br/estatisticas/sociais/educacao/9134-pesquisa-nacional-de-saude-do-escolar.html) do IBGE.  
Base de dados: [PENSE_2015_AMOSTRA2](https://www.ibge.gov.br/estatisticas/downloads-estatisticas.html?caminho=pense/2015/microdados/).

# Importação dos dados

In [1]:
# imports
import numpy as np
import pandas as pd
from zipfile import ZipFile
import requests
import io

In [2]:
# faz download do dataset
url = 'https://github.com/LucasGabrielB/Alura-Bootcamp-Data-Science-Aplicada/raw/main/Modulo-03/datasets/PeNSE_2015_AMOSTRA2.zip'

req = requests.get(url)
zip_file = ZipFile(io.BytesIO(req.content))
zip_file.extractall("datasets")

In [3]:
df = pd.read_csv('/content/datasets/arquivos csv/PENSE_AMOSTRA2_ALUNO.CSV', sep=';', decimal=',')

df.head()

,ANOPESQ,PAIS,REGEOGR,VB00004,VB01001,VB01002,VB01003,VB01004,VB01005,VB01006,VB01007,VB01008A,VB01010A,VB01011,VB01012,VB01013,VB01014,VB01015A,VB01016,VB01017,VB01018,VB01019,VB01020A,VB01021,VB01022,VB01023,VB01024,VB01025,VB01026,VB02001,VB02002,VB02004A,VB02010,VB02011,VB02013,VB02017A,VB02018A,VB02019A,VB02020A,VB02021,...,VB11005,VB11006,VB11007,VB12001,VB12002,VB12003,VB13001,VB13002A,VB13004A,VB13005,VB13006,VB13007,VB13008,VB13009,VB14001,VB14002,VB16001A01,VB16001A02,VB16001A03,VB16001A04,VB16001A05,VB16001A06,VB16001A07,VB16001A08,VB17001,VB17002,VB17003,VB17004,VB17005,VB17006,ESTRATO_EXP,ESTRATOGEOREG,PESO,V0006,V0007,V0008,V0041,aluno,escola,turma
0,2015,76,1,1,2,1,13,7,7,1,2,7,3,2,-1,1,1,1,1,1,2,4,2,4,1,2,2,5,1,8,3,4,3,6,6,1,6,1,-1,2,...,2,1,2,1,1,4,2,-1,3,1,1,-1,1,1,2,2,1,2,2,2,2,2,2,2,371.0,1570.0,37.1,157.0,2,1,1223,1,299.735235,1,2,4,1,1,1,296
1,2015,76,1,1,2,1,14,4,7,1,1,7,3,2,-1,1,1,1,1,1,2,2,2,4,1,2,2,5,3,8,2,1,8,8,6,1,1,5,-1,2,...,2,3,2,4,4,4,1,2,3,3,1,-1,1,1,2,2,2,2,2,1,2,2,2,2,502.0,1520.0,50.2,152.0,2,2,1223,1,355.170081,1,2,4,1,2,1,296
2,2015,76,1,1,1,4,13,6,7,1,1,5,5,2,-1,2,1,1,1,1,2,3,1,4,1,2,2,5,3,2,3,1,8,2,8,1,1,5,5,1,...,2,1,2,1,2,3,1,2,1,2,5,99,1,-1,2,2,1,1,1,1,1,1,1,1,482.0,1614.0,48.2,161.4,2,2,1223,1,299.735235,1,2,4,1,3,1,296
3,2015,76,1,1,1,1,14,6,7,1,1,7,4,2,-1,1,1,1,1,1,2,4,2,4,1,2,2,6,3,2,4,1,3,2,7,1,5,1,-1,2,...,2,1,2,2,2,4,1,10,3,1,2,-1,1,-1,2,2,1,2,2,2,1,2,2,2,694.0,1725.0,69.4,172.5,2,3,1223,1,355.170081,1,2,4,1,4,1,296
4,2015,76,1,1,1,1,13,9,7,1,2,5,4,2,-1,2,1,1,1,1,2,5,1,4,1,2,2,6,5,6,3,5,2,6,2,1,5,1,5,1,...,2,1,4,2,2,4,1,3,2,2,2,13,1,-1,1,2,1,1,1,1,1,1,1,1,789.0,1675.0,78.9,167.5,2,4,1223,1,299.735235,1,2,4,1,5,1,296


In [4]:
df.shape

(16556, 181)

In [5]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16556 entries, 0 to 16555
Columns: 181 entries, ANOPESQ to turma
dtypes: float64(10), int64(171)
memory usage: 22.9 MB


# Analisando os dados

In [6]:
def generate_frequency_table(dataframe: pd.DataFrame, colum_name,
                             convert_index_names :dict=None, index_name=None) -> pd.DataFrame:
    ''' cria uma tabela de frequencias de determinada coluna de um dataframe.

        Parametros

        dataframe : pandas Dataframe.
        colum_name : nome da coluna do dataframe da qual sera gerada a tabela de frequencias.
        convert_index_names: dicionario para renomear o index da tabela de frequencias.
        index_name: nome que sera usado para a coluna index da tabela de frequencias.
    '''

    freq_df = pd.DataFrame({'Frequencia absoluta': dataframe[colum_name].value_counts(),
                            'Frequencia relativa': dataframe[colum_name].value_counts(normalize=True) * 100,
                            'Frequencia acumulada': dataframe[colum_name].value_counts().cumsum(),
                            'Frequencia relativa acumulada': (dataframe[colum_name].value_counts(normalize=True) * 100).cumsum()})
    
    if convert_index_names:
        freq_df.rename(index=convert_index_names, inplace=True)

    if index_name:
        freq_df.rename_axis(index_name, axis=1, inplace=True)

    return freq_df

## Qual seu sexo?

In [7]:
sex_responses_conversion = {
    1: 'Masculino',
    2: 'Feminino'
}

df_sex = generate_frequency_table(df, 'VB01001',
                                  sex_responses_conversion, 'Sexo')

display(df_sex)

Sexo,Frequencia absoluta,Frequencia relativa,Frequencia acumulada,Frequencia relativa acumulada
Masculino,8287,50.054361,8287,50.054361
Feminino,8269,49.945639,16556,100.000000


## Como você se sente em relação ao seu corpo?

In [8]:
body_satisfaction_responses_conversion = {
    1: 'Muito satisfeito',
    2: 'Satisfeito',
    3: 'Indiferente',
    4: 'Insatisfeito',
    5: 'Muito insatisfeito',
    99: 'Não informado'
}

df_body_satisfaction =  generate_frequency_table(df, 'VB11007',
                                                 body_satisfaction_responses_conversion,
                                                 'Sentimento em relação ao corpo')

display(df_body_satisfaction)

Sentimento em relação ao corpo,Frequencia absoluta,Frequencia relativa,Frequencia acumulada,Frequencia relativa acumulada
Satisfeito,6920,41.797536,6920,41.797536
Muito satisfeito,4608,27.832810,11528,69.630345
Insatisfeito,2326,14.049287,13854,83.679633
Indiferente,1807,10.914472,15661,94.594105
Muito insatisfeito,714,4.312636,16375,98.906741
Não informado,181,1.093259,16556,100.000000


### Diferenciando por sexo

Em numeros absolutos:

In [9]:
body_satisfaction_ = pd.crosstab(df['VB11007'], df['VB01001'])
body_satisfaction_.rename(index=body_satisfaction_responses_conversion,
                          columns={1: 'Masculino', 2: 'Feminino'},
                          inplace=True)

display(body_satisfaction_)

VB01001,Masculino,Feminino
VB11007,,
Muito satisfeito,2682,1926
Satisfeito,3490,3430
Indiferente,950,857
Insatisfeito,786,1540
Muito insatisfeito,260,454
Não informado,119,62


Em porcentagens:

In [10]:
body_satisfaction_ = pd.crosstab(df['VB11007'], df['VB01001'], normalize='columns') * 100
body_satisfaction_.rename(index=body_satisfaction_responses_conversion,
                          columns={1: 'Masculino', 2: 'Feminino'},
                          inplace=True)

display(body_satisfaction_)

VB01001,Masculino,Feminino
VB11007,,
Muito satisfeito,32.363944,23.291813
Satisfeito,42.114155,41.480227
Indiferente,11.463738,10.364010
Insatisfeito,9.484735,18.623776
Muito insatisfeito,3.137444,5.490386
Não informado,1.435984,0.749788


## Analisando IMC do aluno

In [11]:
# calculando o IMC do aluno
df['IMC'] = df['VB17003'] / (df['VB17004'] / 100) ** 2
df['IMC']

# adicionando coluna com a classe do IMC no dataframe
bins = [0, 18.5, 25, 30, 35, 40, 100]
labels = ['Abaixo', 'Normal', 'Sobrepeso', 'Obesidade I', 'Obesidade II', 'Obesidade III']

df['CLASSE_IMC'] = pd.cut(x=df['IMC'], bins=bins, labels=labels, include_lowest=True)

df[['IMC', 'CLASSE_IMC']]

,IMC,CLASSE_IMC
0,15.051321,Abaixo
1,21.727839,Normal
2,18.502907,Normal
3,23.322831,Normal
4,28.122076,Sobrepeso
...,...,...
16551,20.113223,Normal
16552,17.930782,Abaixo
16553,23.968489,Normal
16554,18.995343,Normal


In [12]:
# tabela de frequencias do IMC
df_imc = generate_frequency_table(df, 'CLASSE_IMC', index_name='Classe IMC')

display(df_imc)

Classe IMC,Frequencia absoluta,Frequencia relativa,Frequencia acumulada,Frequencia relativa acumulada
Normal,9188,55.496497,9188,55.496497
Abaixo,4975,30.049529,14163,85.546026
Sobrepeso,1760,10.630587,15923,96.176613
Obesidade I,488,2.947572,16411,99.124185
Obesidade II,103,0.622131,16514,99.746316
Obesidade III,42,0.253684,16556,100.000000


### Diferenciando por sexo

Em numeros absolutos:

In [13]:
df_imc_ = pd.crosstab(df['CLASSE_IMC'], df['VB01001'])
df_imc_.rename(index=body_satisfaction_responses_conversion,
                          columns={1: 'Masculino', 2: 'Feminino'},
                          inplace=True)

display(df_imc_)

VB01001,Masculino,Feminino
CLASSE_IMC,,
Abaixo,2620,2355
Normal,4512,4676
Sobrepeso,838,922
Obesidade I,245,243
Obesidade II,47,56
Obesidade III,25,17


Em porcentagens:

In [14]:
df_imc_ = pd.crosstab(df['CLASSE_IMC'], df['VB01001'], normalize='columns') * 100
df_imc_.rename(index=body_satisfaction_responses_conversion,
                          columns={1: 'Masculino', 2: 'Feminino'},
                          inplace=True)

display(df_imc_)

VB01001,Masculino,Feminino
CLASSE_IMC,,
Abaixo,31.615784,28.479865
Normal,54.446724,56.548555
Sobrepeso,10.112224,11.150079
Obesidade I,2.956438,2.938687
Obesidade II,0.567153,0.677228
Obesidade III,0.301677,0.205587


## Analisando altura dos alunos

In [15]:
print(f'Altura\nMin: {df["VB17004"].min()}   Max: {df["VB17004"].max()}')

Altura
Min: 124.1   Max: 195.3


Usando a regra de Sturges para definir o numero de classes.  
Sturges estabeleceu que o número ideal de intervalos k é dado pela expressão:  
  
\begin{equation}
k = 1 + \log_2(N)
\end{equation}
  
– k é o número de classes.  
– N é o número total de observações na amostra.  
– log é o logaritmo comum da base 2.  

In [16]:
def calculate_sturges(N: int) -> int:
    return int(1 + np.log2(N))

In [17]:
# calculando valor de k
k = calculate_sturges(len(df))

print('k =', k)

k = 15


In [18]:
# adicionando coluna com a classe da altura no dataframe
df['CLASSE_ALTURA'] = pd.cut(x=df['VB17004'], bins=k, include_lowest=True)

df_heights = generate_frequency_table(df, colum_name='CLASSE_ALTURA', index_name='Altura')

display(df_heights)

Altura,Frequencia absoluta,Frequencia relativa,Frequencia acumulada,Frequencia relativa acumulada
"(157.327, 162.073]",3110,18.784731,3110,18.784731
"(152.58, 157.327]",2734,16.513651,5844,35.298381
"(162.073, 166.82]",2550,15.402271,8394,50.700652
"(166.82, 171.567]",2063,12.460739,10457,63.161392
"(147.833, 152.58]",1782,10.763469,12239,73.924861
"(171.567, 176.313]",1416,8.552791,13655,82.477652
"(143.087, 147.833]",1012,6.112588,14667,88.590239
"(176.313, 181.06]",749,4.524040,15416,93.114279
"(138.34, 143.087]",484,2.923411,15900,96.037690
"(181.06, 185.807]",300,1.812032,16200,97.849722


## Analisando peso dos alunos

In [19]:
print(f'Peso\nMin: {df["VB17003"].min()}   Max: {df["VB17003"].max()}')

Peso
Min: 0.8   Max: 171.5


In [20]:
# adicionando coluna com a classe do peso no dataframe
df['CLASSE_PESO'] = pd.cut(x=df['VB17003'], bins=k, include_lowest=True)

df_weights = generate_frequency_table(df, colum_name='CLASSE_PESO', index_name='Peso')

display(df_weights)

Peso,Frequencia absoluta,Frequencia relativa,Frequencia acumulada,Frequencia relativa acumulada
"(46.32, 57.7]",5754,34.754772,5754,34.754772
"(34.94, 46.32]",3852,23.266489,9606,58.021261
"(57.7, 69.08]",3724,22.493356,13330,80.514617
"(69.08, 80.46]",1425,8.607151,14755,89.121769
"(23.56, 34.94]",927,5.599179,15682,94.720947
"(80.46, 91.84]",544,3.285818,16226,98.006765
"(91.84, 103.22]",202,1.220101,16428,99.226866
"(103.22, 114.6]",77,0.465088,16505,99.691955
"(114.6, 125.98]",20,0.120802,16525,99.812757
"(125.98, 137.36]",15,0.090602,16540,99.903358


## Analisando uso de drogas

Idade que usou drogas pela primeira vez, separado por sexo, em porcentagens em relação ao total do sexo.

In [21]:
drugs_age_responses_conversion = {
    -1:	'Pulo no questionário',
    7: '7 anos ou menos',
    8: '8 anos',
    9: '9 anos',
    10:	'10 anos',
    11:	'11 anos',
    12:	'12 anos',
    13:	'13 anos',
    14:	'14 anos',
    15:	'15 anos',
    16:	'16 anos',
    17:	'17 anos',
    18:	'18 anos ou mais',
    99:	'Não informado'
}

# removebndo pessoas que nao responderam ou pularam a questao
df_temp = df.query('VB06002 != -1 and VB06002 != 99')

df_drugs = pd.crosstab(df_temp['VB06002'], df_temp['VB01001'], normalize='columns') * 100

df_drugs.rename(index=drugs_age_responses_conversion, columns={1: 'Masculino', 2: 'Feminino'}, inplace=True)
df_drugs.rename_axis('Idade', inplace=True)

display(df_drugs)

VB01001,Masculino,Feminino
Idade,,
7 anos ou menos,3.448276,1.415094
8 anos,0.638570,0.314465
9 anos,0.510856,0.314465
10 anos,1.915709,0.628931
11 anos,3.703704,2.358491
12 anos,6.130268,10.691824
13 anos,15.325670,19.025157
14 anos,21.200511,20.440252
15 anos,22.094508,23.427673
